Getting down to business. We're going to extract ten tables from 10 PDFs and then convert them to CSVs. At the end, we'll pull it all together into one big CSV.

In [50]:
#Load libraries

import pdfplumber
import pandas as pd
import numpy as np
from pathlib import Path
import re
import fitz
from PyPDF2 import PdfFileReader, PdfFileWriter

In [69]:
#Bring in the PDFs

pdf_path22 = "mcps2022.pdf"
pdf22 = pdfplumber.open(pdf_path22)
#pages of interest: 458, 459, 460, 464, 466

pdf_path21 = "mcps2021.pdf"
pdf21 = pdfplumber.open(pdf_path21)
#pages of interest: 458, 459, 460, 464, 466

pdf_path20 = "mcps2020.pdf"
pdf20 = pdfplumber.open(pdf_path20)
#pages of interest: 458, 459, 460, 464, 466 

pdf_path19 = "mcps2019.pdf"
pdf19 = pdfplumber.open(pdf_path19)
#pages of interest: 456, 457, 458, 462, 464 

pdf_path18 = "mcps2018.pdf"
pdf18 = pdfplumber.open(pdf_path18)
#pages of interest: 454, 455, 456, 460, 462

pdf_path17 = "mcps2017.pdf"
pdf17 = pdfplumber.open(pdf_path17)
#pages of interest: 452, 453, 454, 458, 460

pdf_path16 = "mcps2016.pdf"
pdf16 = pdfplumber.open(pdf_path16)
#pages of interest: 452, 453, 454, 458, 460

pdf_path15 = "mcps2015.pdf"
pdf15 = pdfplumber.open(pdf_path15)
#pages of interest: 452, 453, 454, 458, 460 (?)

pdf_path14 = "mcps2014.pdf"
pdf14 = pdfplumber.open(pdf_path14)


To extract the data, I made use of a tutorial I found here: 
https://medium.com/@karthickrajm/how-to-extract-table-from-pdf-using-python-pdfplumber-a2010b184431

I'm including inline comments for the first extraction. The rest of the code is quite similar, so I will exclude comments when the line of code's premise was already described.

In [102]:
# Get all 2022 data

#Target the table on one page
tbl2022_es1 = pdf22.pages[458].extract_table()
schools_es1 = pd.DataFrame(tbl2022_es1[1::],columns=tbl2022_es1[0])

#Reset column names
schools_es1.columns = schools_es1.iloc[0]
schools_es1 = schools_es1[1:]

#This will help when we include other schools in a bigger dataframe.
schools_es1['Level']='Elementary School' 

#p2
tbl2022_es2 = pdf22.pages[459].extract_table()
schools_es2 = pd.DataFrame(tbl2022_es2[1::],columns=tbl2022_es2[0])
schools_es2.columns = schools_es2.iloc[0]
schools_es2 = schools_es2[1:]
schools_es2['Level']='Elementary School' 

#p3
tbl2022_es3 = pdf22.pages[460].extract_table()
schools_es3 = pd.DataFrame(tbl2022_es3[1::],columns=tbl2022_es3[0])
schools_es3.columns = schools_es3.iloc[0]
schools_es3 = schools_es3[1:]
schools_es3['Level']='Elementary School' 

#p4
tbl2022_ms = pdf22.pages[464].extract_table()
schools_ms22 = pd.DataFrame(tbl2022_ms[1::],columns=tbl2022_ms[0])
schools_ms22.columns = schools_ms22.iloc[0]
schools_ms22 = schools_ms22[1:]
schools_ms22['Level']='Middle School' 

#p5
tbl2022_hs = pdf22.pages[466].extract_table()
schools_hs22 = pd.DataFrame(tbl2022_hs[1::],columns=tbl2022_hs[0])
schools_hs22.columns = schools_hs22.iloc[0]
schools_hs22 = schools_hs22[1:]
schools_hs22['Level']='High School' 

# Concatenate tables from each page
mcps_22_all = pd.concat([schools_es1, schools_es2, schools_es3, schools_ms22, schools_hs22], ignore_index=False)
mcps_22_all = mcps_22_all.reset_index(drop=True)

# Rename columns to match later large table and to make easier for analysis later.
mcps_22_all.columns = mcps_22_all.columns.str.replace('School Name', 'school')
mcps_22_all.columns = mcps_22_all.columns.str.replace('Spec. Ed.', 'special_ed')
mcps_22_all['Year']='2021-2022'
mcps_22_all

C:\Users\Rina\AppData\Local\Temp\ipykernel_6104\2304647381.py:80: FutureWarning: The default value of regex will change from True to False in a future version.
  mcps_22_all.columns = mcps_22_all.columns.str.replace('Spec. Ed.', 'special_ed')


,school,Total,Male,Female,X,AM,AS,BL,HI,PI,MU,WH,EML,FARMS,special_ed,Level,Year
0,Arcola ES,631,51.2,48.8,≤5.0,≤5.0,6.0,18.7,69.3,≤5.0,≤5.0,≤5.0,49.0,77.0,7.9,Elementary School,2021-2022
1,Ashburton ES,834,48.7,51.3,≤5.0,≤5.0,18.3,16.2,18.0,≤5.0,9.5,37.6,15.9,12.4,9.5,Elementary School,2021-2022
2,Bannockburn ES,421,48.7,51.3,≤5.0,≤5.0,14.5,6.2,11.4,≤5.0,7.8,60.1,≤5.0,≤5.0,8.8,Elementary School,2021-2022
3,Lucy V. Barnsley ES,689,53.6,46.4,≤5.0,≤5.0,11.2,13.6,35.7,≤5.0,7.8,30.8,18.3,34.4,12.6,Elementary School,2021-2022
4,Beall ES,497,52.1,47.9,≤5.0,≤5.0,16.7,12.5,23.5,≤5.0,7.6,39.6,11.1,27.6,11.9,Elementary School,2021-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Springbrook HS,1681,57.9,41.9,≤5.0,≤5.0,10.4,38.4,42.7,≤5.0,≤5.0,5.7,18.6,59.0,7.9,High School,2021-2022
197,Watkins Mill HS,1594,54.5,45.4,≤5.0,≤5.0,9.8,23.5,58.6,≤5.0,≤5.0,5.3,28.4,67.9,13.4,High School,2021-2022
198,Wheaton HS,2375,59.2,40.7,≤5.0,≤5.0,11.3,19.6,56.2,≤5.0,≤5.0,10.3,18.4,58.2,9.7,High School,2021-2022
199,Walt Whitman HS,2029,52.1,47.8,≤5.0,≤5.0,14.3,≤5.0,11.2,≤5.0,7.5,63.0,≤5.0,≤5.0,9.5,High School,2021-2022


In [104]:
mcps_22_all.to_csv('2022_mcps_data_new.csv', index=False)

In [103]:
# Get all 2021 data

tbl2021_es1 = pdf21.pages[458].extract_table()
schools21_es1 = pd.DataFrame(tbl2021_es1[1::],columns=tbl2021_es1[0])
schools21_es1.columns = schools21_es1.iloc[0]
schools21_es1 = schools21_es1[1:]
schools21_es1['Level']='Elementary School' 

#p2
tbl2021_es2 = pdf21.pages[459].extract_table()
schools21_es2 = pd.DataFrame(tbl2021_es2[1::],columns=tbl2021_es2[0])
schools21_es2.columns = schools21_es2.iloc[0]
schools21_es2 = schools21_es2[1:]
schools21_es2['Level']='Elementary School' 

#p3
tbl2021_es3 = pdf21.pages[460].extract_table()
schools21_es3 = pd.DataFrame(tbl2021_es3[1::],columns=tbl2021_es3[0])
schools21_es3.columns = schools21_es3.iloc[0]
schools21_es3 = schools21_es3[1:]
schools21_es3['Level']='Elementary School' 

#p4
tbl2021_ms = pdf21.pages[464].extract_table()
schools_ms21 = pd.DataFrame(tbl2021_ms[1::],columns=tbl2021_ms[0])
schools_ms21.columns = schools_ms21.iloc[0]
schools_ms21 = schools_ms21[1:]
schools_ms21['Level']='Middle School' 

#p5
tbl2021_hs = pdf21.pages[466].extract_table()
schools_hs21 = pd.DataFrame(tbl2021_hs[1::],columns=tbl2021_hs[0])
schools_hs21.columns = schools_hs21.iloc[0]
schools_hs21 = schools_hs21[1:]
schools_hs21['Level']='High School' 

mcps_21_all = pd.concat([schools21_es1, schools21_es2, schools21_es3, schools_ms21, schools_hs21], ignore_index=False)
mcps_21_all = mcps_21_all.reset_index(drop=True)

mcps_21_all['Year']='2020-2021'
mcps_21_all.columns = mcps_21_all.columns.str.replace('School Name', 'school')
mcps_21_all.columns = mcps_21_all.columns.str.replace('Spec. Ed.', 'special_ed')
mcps_21_all

C:\Users\Rina\AppData\Local\Temp\ipykernel_6104\1208462636.py:81: FutureWarning: The default value of regex will change from True to False in a future version.
  mcps_21_all.columns = mcps_21_all.columns.str.replace('Spec. Ed.', 'special_ed')


,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year
0,Arcola ES,703,51.2,48.8,≤5.0,5.4,19.1,69.0,≤5.0,≤5.0,≤5.0,43.0,78.0,8.0,Elementary School,2020-2021
1,Ashburton ES,900,50.4,49.4,≤5.0,17.7,15.9,18.0,≤5.0,38.1,9.7,13.7,11.9,7.6,Elementary School,2020-2021
2,Bannockburn ES,401,50.9,49.1,≤5.0,12.5,≤5.0,10.0,≤5.0,64.1,8.5,≤5.0,≤5.0,9.2,Elementary School,2020-2021
3,Lucy V. Barnsley ES,701,54.9,45.1,≤5.0,12.6,12.7,35.1,≤5.0,30.5,8.1,14.1,33.7,12.6,Elementary School,2020-2021
4,Beall ES,516,50.6,49.4,≤5.0,14.3,15.9,22.1,≤5.0,39.5,7.9,12.8,30.6,13.4,Elementary School,2020-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Springbrook HS,1694,58.9,41.1,≤5.0,12.9,39.0,39.3,≤5.0,6.1,≤5.0,17.2,56.2,8.1,High School,2020-2021
197,Watkins Mill HS,1597,54.2,45.8,≤5.0,9.6,26.2,54.9,≤5.0,6.4,≤5.0,23.4,62.0,13.3,High School,2020-2021
198,Wheaton HS,2280,59.0,41.0,≤5.0,11.8,20.2,55.9,≤5.0,9.5,≤5.0,17.2,55.5,8.5,High School,2020-2021
199,Walt Whitman HS,1991,52.2,47.7,≤5.0,14.0,≤5.0,10.3,≤5.0,64.5,6.9,≤5.0,≤5.0,9.4,High School,2020-2021


In [105]:
mcps_21_all.to_csv('2021_mcps_data_new.csv', index=False)

In [106]:
# Get all 2020 data

tbl2020_es1 = pdf20.pages[458].extract_table()
schools20_es1 = pd.DataFrame(tbl2020_es1[1::],columns=tbl2020_es1[0])
schools20_es1.columns = schools20_es1.iloc[0]
schools20_es1 = schools20_es1[1:]
schools20_es1['Level']='Elementary School' 

#p2
tbl2020_es2 = pdf20.pages[459].extract_table()
schools20_es2 = pd.DataFrame(tbl2020_es2[1::],columns=tbl2020_es2[0])
schools20_es2.columns = schools20_es2.iloc[0]
schools20_es2 = schools20_es2[1:]
schools20_es2['Level']='Elementary School' 

#p3
tbl2020_es3 = pdf20.pages[460].extract_table()
schools20_es3 = pd.DataFrame(tbl2020_es3[1::],columns=tbl2020_es3[0])
schools20_es3 .columns = schools20_es3 .iloc[0]
schools20_es3  = schools20_es3[1:]
schools20_es3['Level']='Elementary School' 

#p4
tbl2020_ms = pdf20.pages[464].extract_table()
schools_ms20 = pd.DataFrame(tbl2020_ms[1::],columns=tbl2020_ms[0])
schools_ms20.columns = schools_ms20.iloc[0]
schools_ms20 = schools_ms20[1:]
schools_ms20['Level']='Middle School' 

#p5
tbl2020_hs = pdf20.pages[466].extract_table()
schools_hs20 = pd.DataFrame(tbl2020_hs[1::],columns=tbl2020_hs[0])
schools_hs20.columns = schools_hs20.iloc[0]
schools_hs20 = schools_hs20[1:]
schools_hs20['Level']='High School' 


mcps_20_all = pd.concat([schools20_es1, schools20_es2, schools20_es3, schools_ms20, schools_hs20], ignore_index=False)
mcps_20_all = mcps_20_all.reset_index(drop=True)
mcps_20_all['Year']='2019-2020'
mcps_20_all.columns = mcps_20_all.columns.str.replace('School Name', 'school')
mcps_20_all.columns = mcps_20_all.columns.str.replace('Spec. Ed.', 'special_ed')
mcps_20_all

C:\Users\Rina\AppData\Local\Temp\ipykernel_6104\1625488621.py:81: FutureWarning: The default value of regex will change from True to False in a future version.
  mcps_20_all.columns = mcps_20_all.columns.str.replace('Spec. Ed.', 'special_ed')


,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year
0,Arcola ES,748,50.7,49.3,≤5.0,5.6,19.8,69.3,≤5.0,≤5.0,≤5.0,49.3,74.7,10.7,Elementary School,2019-2020
1,Ashburton ES,922,50.1,49.9,≤5.0,17.4,16.6,17.4,≤5.0,39.3,8.8,15.3,11.1,7.8,Elementary School,2019-2020
2,Bannockburn ES,461,51.4,48.6,≤5.0,11.3,≤5.0,10.8,≤5.0,66.6,6.5,5.4,≤5.0,7.8,Elementary School,2019-2020
3,Lucy V. Barnsley ES,737,54.8,45.2,≤5.0,11.9,12.6,34.5,≤5.0,33.0,7.7,15.1,30.1,13.8,Elementary School,2019-2020
4,Beall ES,531,53.3,46.7,≤5.0,13.4,15.3,25.0,≤5.0,38.2,7.7,17.1,30.1,12.6,Elementary School,2019-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Springbrook HS,1746,57.4,42.6,≤5.0,12.4,38.1,40.5,≤5.0,6.1,≤5.0,19.0,48.5,8.6,High School,2019-2020
197,Watkins Mill HS,1590,53.0,47.0,≤5.0,8.6,26.7,54.7,≤5.0,6.7,≤5.0,24.8,51.3,12.8,High School,2019-2020
198,Wheaton HS,2179,60.0,40.0,≤5.0,11.2,19.9,57.3,≤5.0,9.3,≤5.0,22.3,48.0,8.6,High School,2019-2020
199,Walt Whitman HS,2039,50.4,49.6,≤5.0,13.9,≤5.0,8.8,≤5.0,67.0,6.2,≤5.0,≤5.0,8.3,High School,2019-2020


In [107]:
mcps_20_all.to_csv('2020_mcps_data_new.csv', index=False)

In [108]:
# Get all 2019 data

tbl2019_es1 = pdf19.pages[456].extract_table()
schools19_es1 = pd.DataFrame(tbl2019_es1[1::],columns=tbl2019_es1[0])
schools19_es1.columns = schools19_es1.iloc[0]
schools19_es1 = schools19_es1[1:]
schools19_es1['Level']='Elementary School' 

#p2
tbl2019_es2 = pdf19.pages[457].extract_table()
schools19_es2 = pd.DataFrame(tbl2019_es2[1::],columns=tbl2019_es2[0])
schools19_es2.columns = schools19_es2.iloc[0]
schools19_es2 = schools19_es2[1:]
schools19_es2['Level']='Elementary School' 

#p3
tbl2019_es3 = pdf19.pages[458].extract_table()
schools19_es3 = pd.DataFrame(tbl2019_es3 [1::],columns=tbl2019_es3 [0])
schools19_es3.columns = schools19_es3.iloc[0]
schools19_es3 = schools19_es3[1:]
schools19_es3['Level']='Elementary School' 

#p4
tbl2019_ms = pdf19.pages[462].extract_table()
schools_ms19 = pd.DataFrame(tbl2019_ms[1::],columns=tbl2019_ms[0])
schools_ms19.columns = schools_ms19.iloc[0]
schools_ms19 = schools_ms19[1:]
schools_ms19['Level']='Middle School' 

#p5
tbl2019_hs = pdf19.pages[464].extract_table()
schools_hs19 = pd.DataFrame(tbl2019_hs[1::],columns=tbl2019_hs[0])
schools_hs19.columns = schools_hs19.iloc[0]
schools_hs19 = schools_hs19[1:]
schools_hs19['Level']='High School' 


mcps_19_all = pd.concat([schools19_es1, schools19_es2, schools19_es3, schools_ms19, schools_hs19], ignore_index=False)

mcps_19_all = mcps_19_all.reset_index(drop=True)
mcps_19_all['Year']='2018-2019'
mcps_19_all.columns = mcps_19_all.columns.str.replace('School Name', 'school')
mcps_19_all.columns = mcps_19_all.columns.str.replace('Spec. Ed.', 'special_ed')
mcps_19_all

C:\Users\Rina\AppData\Local\Temp\ipykernel_6104\3965676031.py:81: FutureWarning: The default value of regex will change from True to False in a future version.
  mcps_19_all.columns = mcps_19_all.columns.str.replace('Spec. Ed.', 'special_ed')


,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year
0,Arcola ES,732,48.4,51.6,≤5.0,7.4,19.0,68.6,≤5.0,≤5.0,≤5.0,47.1,75.1,9.6,Elementary School,2018-2019
1,Ashburton ES,891,49.6,50.4,≤5.0,16.8,15.2,17.6,≤5.0,40.3,9.7,15.5,11.4,8.9,Elementary School,2018-2019
2,Bannockburn ES,440,53.0,47.0,≤5.0,12.7,≤5.0,11.6,≤5.0,63.2,8.2,8.6,≤5.0,6.6,Elementary School,2018-2019
3,Lucy V. Barnsley ES,721,57.0,43.0,≤5.0,14.0,9.8,33.8,≤5.0,35.1,6.9,13.7,26.9,15.1,Elementary School,2018-2019
4,Beall ES,572,54.5,45.5,≤5.0,15.7,16.1,22.4,≤5.0,39.2,6.5,17.5,30.1,16.8,Elementary School,2018-2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Springbrook HS,1735,59.7,40.3,≤5.0,12.6,37.8,39.8,≤5.0,6.7,≤5.0,18.7,48.0,7.8,High School,2018-2019
196,Watkins Mill HS,1615,52.9,47.1,≤5.0,8.2,29.4,51.1,≤5.0,7.2,≤5.0,22.2,50.0,10.1,High School,2018-2019
197,Wheaton HS,2077,59.4,40.6,≤5.0,12.1,22.3,53.9,≤5.0,9.3,≤5.0,20.9,46.9,6.1,High School,2018-2019
198,Walt Whitman HS,2098,52.0,48.0,≤5.0,14.1,≤5.0,8.1,≤5.0,68.5,5.6,≤5.0,≤5.0,7.5,High School,2018-2019


In [109]:
mcps_19_all.to_csv('2019_mcps_data_new.csv', index=False)

#Get all 2018 data.

The method above did not work with 2018 data. It was skipping a column. I ran it through Tabula. 
Here, we can clean it up to match the format of others.

In [112]:
# Get all 2018 data

mcps_2018_all = pd.read_csv('mcps2018_tabula.csv')
mcps_2018_all.columns = mcps_2018_all.columns.str.replace('School Name', 'school')
mcps_2018_all['Level'] = pd.Series(dtype='object')
mcps_2018_all['Year']='2017-2018'
mcps_2018_all.columns = mcps_2018_all.columns.str.replace('SPED', 'special_ed')

mcps_2018_all.loc[mcps_2018_all['school'].str.contains('ES'), 'Level'] = 'Elementary School'
mcps_2018_all.loc[mcps_2018_all['school'].str.contains('HS'), 'Level'] = 'High School'
mcps_2018_all.loc[mcps_2018_all['school'].str.contains('MS'), 'Level'] = 'Middle School'
   

mcps_2018_all

,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year
0,Arcola ES,686,49.0,51.0,≤5.0,7.7,18.5,68.5,≤5.0,≤5.0,≤5.0,48.4,76.4,11.1,Elementary School,2017-2018
1,Ashburton ES,877,50.6,49.4,≤5.0,16,15.5,15.6,≤5.0,42.5,9.9,12.9,12.7,9.6,Elementary School,2017-2018
2,Bannockburn ES,449,51.2,48.8,≤5.0,12.9,≤5.0,8.7,≤5.0,67.5,6.9,8.5,≤5.0,6,Elementary School,2017-2018
3,Lucy V. Barnsley ES,724,56.6,43.4,≤5.0,13.7,10.1,33,≤5.0,34.1,8.7,14.4,29.1,14.5,Elementary School,2017-2018
4,Beall ES,783,53.1,46.9,≤5.0,24.6,11.5,23.6,≤5.0,34.2,5.6,18.4,25,9.8,Elementary School,2017-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Springbrook HS,1796,59.9,40.1,≤5.0,12.8,38,38.5,≤5.0,7.2,≤5.0,17.7,45.6,8.9,High School,2017-2018
195,Watkins Mill HS,1660,52.6,47.4,≤5.0,8,29.8,50.2,≤5.0,7.7,≤5.0,24.2,50.2,11.9,High School,2017-2018
196,Wheaton HS,1958,59.8,40.2,≤5.0,12.2,22.6,54.7,≤5.0,8.5,≤5.0,21.1,49.7,8.8,High School,2017-2018
197,Walt Whitman HS,2085,51.7,48.3,≤5.0,15.2,≤5.0,8.7,≤5.0,66.8,≤5.0,≤5.0,≤5.0,9.1,High School,2017-2018


In [113]:
mcps_2018_all.to_csv('2018_mcps_data_new.csv', index=False)

After some glitches with 2018 data, we ran into the same issue with 2017 data. We'll repeat what we done with 2018 date here. That means, first Tabula, then some light cleaning.

In [114]:
# Get all 2017 data 
mcps_2017_all = pd.read_csv('mcps2017_tabula.csv')
mcps_2017_all.columns = mcps_2017_all.columns.str.replace('School Name', 'school')
mcps_2017_all['Level'] = pd.Series(dtype='object')
mcps_2017_all['Year']='2016-2017'
mcps_2017_all.columns = mcps_2017_all.columns.str.replace('SPED', 'special_ed')

mcps_2017_all.loc[mcps_2017_all['school'].str.contains('ES'), 'Level'] = 'Elementary School'
mcps_2017_all.loc[mcps_2017_all['school'].str.contains('HS'), 'Level'] = 'High School'
mcps_2017_all.loc[mcps_2017_all['school'].str.contains('MS'), 'Level'] = 'Middle School'
   

mcps_2017_all

,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year
0,Arcola ES,704,48.7,51.3,≤5.0,8.4,18.6,68.2,≤5.0,≤5.0,≤5.0,44.9,76.8,10.4,Elementary School,2016-2017
1,Ashburton ES,905,51.3,48.7,≤5.0,15.0,14.9,16.1,≤5.0,43.8,9.7,11.6,12.5,12.8,Elementary School,2016-2017
2,Bannockburn ES,454,51.5,48.5,≤5.0,11.7,≤5.0,8.8,≤5.0,67.8,8.1,6.8,≤5.0,7.7,Elementary School,2016-2017
3,Lucy V. Barnsley ES,712,55.8,44.2,≤5.0,14.0,10.0,34.1,≤5.0,33.8,7.7,13.2,31.0,14.2,Elementary School,2016-2017
4,Beall ES,812,54.6,45.4,≤5.0,24.0,12.1,21.9,≤5.0,35.2,6.4,17.5,25.2,10.7,Elementary School,2016-2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Springbrook HS,1792,58.1,41.9,≤5.0,11.5,39.1,37.8,≤5.0,7.8,≤5.0,15.3,47.3,9.5,High School,2016-2017
194,Watkins Mill HS,1584,55.2,44.8,≤5.0,8.0,31.4,46.6,≤5.0,9.3,≤5.0,21.8,53.4,12.5,High School,2016-2017
195,Wheaton HS,1762,60.1,39.9,≤5.0,12.6,24.7,52.3,≤5.0,8.2,≤5.0,19.1,49.4,10.8,High School,2016-2017
196,Walt Whitman HS,2055,50.0,50.0,≤5.0,14.3,≤5.0,9.6,≤5.0,67.5,≤5.0,≤5.0,≤5.0,9.1,High School,2016-2017


In [115]:
mcps_2017_all.to_csv('2017_mcps_data_new.csv', index=False)

In [116]:
# Get all 2016 data
mcps_2016_all = pd.read_csv('mcps2016_tabula.csv')
mcps_2016_all.columns = mcps_2016_all.columns.str.replace('School Name', 'school')
mcps_2016_all['Level'] = pd.Series(dtype='object')
mcps_2016_all['Year']='2015-2016'
mcps_2016_all.columns = mcps_2016_all.columns.str.replace('SPED', 'special_ed')

mcps_2016_all.loc[mcps_2017_all['school'].str.contains('ES'), 'Level'] = 'Elementary School'
mcps_2016_all.loc[mcps_2017_all['school'].str.contains('HS'), 'Level'] = 'High School'
mcps_2016_all.loc[mcps_2017_all['school'].str.contains('MS'), 'Level'] = 'Middle School'
   

mcps_2016_all

,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year
0,Arcola ES,693,50.8,49.2,≤5.0,8.5,18.2,69.7,≤5.0,≤5.0,≤5.0,46,74.7,10.5,Elementary School,2015-2016
1,Ashburton ES,907,52.0,48.0,≤5.0,16.2,14.8,14.0,≤5.0,45,9.4,14.9,11.8,12.8,Elementary School,2015-2016
2,Bannockburn ES,420,53.1,46.9,≤5.0,11,≤5.0,10.0,≤5.0,67.4,9,11,≤5.0,≤5.0,Elementary School,2015-2016
3,Lucy V. Barnsley ES,710,57.7,42.3,≤5.0,13.7,9.6,31.5,≤5.0,38.2,7,13.4,27.6,15.2,Elementary School,2015-2016
4,Beall ES,824,53.9,46.1,≤5.0,22.5,13.5,21.4,≤5.0,35.6,6.7,17.5,27.3,9.2,Elementary School,2015-2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Springbrook HS,1790,57.6,42.4,≤5.0,11.3,41.8,35.1,≤5.0,8,≤5.0,14.9,44,9.7,High School,2015-2016
193,Watkins Mill HS,1524,55.3,44.7,≤5.0,8.7,31.6,43.1,≤5.0,11.7,≤5.0,20,51.4,12.9,High School,2015-2016
194,Wheaton HS,1554,59.2,40.8,≤5.0,12.1,26.4,50.7,≤5.0,8.4,≤5.0,18.9,47.6,11.1,High School,2015-2016
195,Walt Whitman HS,1977,49.5,50.5,≤5.0,13.2,≤5.0,9.3,≤5.0,69,≤5.0,8.8,≤5.0,9.5,High School,2015-2016


In [117]:
mcps_2016_all.to_csv('2016_mcps_data_new.csv', index=False)

In [118]:
# 2015 time

mcps_2015_all = pd.read_csv('mcps2015_tabula.csv')
mcps_2015_all.columns = mcps_2015_all.columns.str.replace('School Name', 'school')
mcps_2015_all['Level'] = pd.Series(dtype='object')
mcps_2015_all['Year']='2014-2015'
mcps_2015_all.columns = mcps_2015_all.columns.str.replace('SPED', 'special_ed')

mcps_2015_all.loc[mcps_2015_all['school'].str.contains('ES'), 'Level'] = 'Elementary School'
mcps_2015_all.loc[mcps_2015_all['school'].str.contains('HS'), 'Level'] = 'High School'
mcps_2015_all.loc[mcps_2015_all['school'].str.contains('MS'), 'Level'] = 'Middle School'
   

mcps_2015_all


,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year
0,Arcola ES,725,52.6,47.4,≤5.0,7.9,18.2,69.1,≤5.0,≤5.0,≤5.0,43.6,74.9,9.5,Elementary School,2014-2015
1,Ashburton ES,899,50.2,49.8,≤5.0,15.5,14.1,13.6,≤5.0,47.6,9.0,10.9,12.6,12.7,Elementary School,2014-2015
2,Bannockburn ES,407,51.8,48.2,≤5.0,11.5,≤5.0,9.1,≤5.0,66.8,9.6,9.8,≤5.0,≤5.0,Elementary School,2014-2015
3,Lucy V. Barnsley ES,686,58.2,41.8,≤5.0,14.1,10.2,30.6,≤5.0,38.8,6.3,14.0,29.6,14.7,Elementary School,2014-2015
4,Beall ES,800,55.0,45.0,≤5.0,21.6,13.9,22.3,≤5.0,34.6,7.5,14.9,26.1,9.9,Elementary School,2014-2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Springbrook HS,1747,56.3,43.7,≤5.0,11.2,41.9,35.7,≤5.0,8.2,≤5.0,10.2,45.2,10.3,High School,2014-2015
193,Watkins Mill HS,1492,53.2,46.8,≤5.0,9.9,34.9,40.0,≤5.0,11.0,≤5.0,13.7,53.8,11.8,High School,2014-2015
194,Wheaton HS,1465,58.0,42.0,≤5.0,10.0,24.8,54.5,≤5.0,8.3,≤5.0,17.1,54.4,11.8,High School,2014-2015
195,Walt Whitman HS,1902,49.6,50.4,≤5.0,12.2,≤5.0,9.0,≤5.0,69.5,5.6,≤5.0,≤5.0,9.8,High School,2014-2015


In [119]:
mcps_2015_all.to_csv('2015_mcps_data_new.csv', index=False)

In [120]:
# 2014 time

mcps_2014_all = pd.read_csv('mcps2014_tabula.csv')
mcps_2014_all.columns = mcps_2014_all.columns.str.replace('School Name', 'school')
mcps_2014_all['Level'] = pd.Series(dtype='object')
mcps_2014_all['Year']='2013-2014'
mcps_2014_all.columns = mcps_2014_all.columns.str.replace('SPED', 'special_ed')

mcps_2014_all.loc[mcps_2014_all['school'].str.contains('ES'), 'Level'] = 'Elementary School'
mcps_2014_all.loc[mcps_2014_all['school'].str.contains('HS'), 'Level'] = 'High School'
mcps_2014_all.loc[mcps_2014_all['school'].str.contains('MS'), 'Level'] = 'Middle School'
   

mcps_2014_all

,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year
0,Arcola ES,708,52.0,48.0,≤5.0,8.3,17.5,68.5,≤5.0,≤5.0,≤5.0,44.8,75.6,9.0,Elementary School,2013-2014
1,Ashburton ES,837,51.3,48.7,≤5.0,14.7,13.0,12.8,≤5.0,50.9,8.4,10.6,11.4,14.0,Elementary School,2013-2014
2,Bannockburn ES,398,51.8,48.2,≤5.0,10.8,≤5.0,7.5,≤5.0,68.1,10.8,10.8,≤5.0,5.8,Elementary School,2013-2014
3,Lucy V. Barnsley ES,681,57.1,42.9,≤5.0,15.3,10.4,29.1,≤5.0,39.2,6.0,13.1,28.2,13.5,Elementary School,2013-2014
4,Beall ES,782,54.7,45.3,≤5.0,25.1,13.4,19.1,≤5.0,34.4,7.8,15.6,25.4,8.1,Elementary School,2013-2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Springbrook HS,1749,56.0,44.0,≤5.0,11.6,42.3,33.7,≤5.0,9.3,≤5.0,8.9,44.0,10.6,High School,2013-2014
193,Watkins Mill HS,1449,52.6,47.4,≤5.0,9.6,37.5,36.0,≤5.0,12.6,≤5.0,9.1,50.1,12.9,High School,2013-2014
194,Wheaton HS,1336,57.6,42.4,≤5.0,10.6,25.6,53.7,≤5.0,7.8,≤5.0,15.4,58.9,12.6,High School,2013-2014
195,Walt Whitman HS,1921,51.5,48.5,≤5.0,12.3,≤5.0,9.0,≤5.0,70.1,5.3,≤5.0,≤5.0,11.8,High School,2013-2014


In [121]:
mcps_2014_all.to_csv('2014_mcps_data_new.csv', index=False)

In [125]:
# 2013 time

mcps_2013_all = pd.read_csv('mcps2013_tabula.csv')
mcps_2013_all.columns = mcps_2013_all.columns.str.replace('School Name', 'school')
mcps_2013_all['Level'] = pd.Series(dtype='object')
mcps_2013_all['Year']='2012-2013'
mcps_2013_all.columns = mcps_2013_all.columns.str.replace('SPED', 'special_ed')

mcps_2013_all.loc[mcps_2013_all['school'].str.contains('ES'), 'Level'] = 'Elementary School'
mcps_2013_all.loc[mcps_2013_all['school'].str.contains('HS'), 'Level'] = 'High School'
mcps_2013_all.loc[mcps_2013_all['school'].str.contains('MS'), 'Level'] = 'Middle School'
   
mcps_2013_all.to_csv('2013_mcps_data_new.csv', index=False)

In [126]:
mcps_all = pd.concat([mcps_2013_all, mcps_2014_all, mcps_2015_all, mcps_2016_all, mcps_2017_all, mcps_2018_all, mcps_19_all, mcps_20_all, mcps_21_all, mcps_22_all], ignore_index=False)

In [127]:
mcps_all

,school,Total,Male,Female,AM,AS,BL,HI,PI,WH,MU,ESOL,FARMS,special_ed,Level,Year,X,EML
0,Arcola ES,713,53.2,46.8,≤5.0,10.0,18.0,65.4,≤5.0,≤5.0,≤5.0,50.6,75.6,10.2,Elementary School,2012-2013,NaN,NaN
1,Ashburton ES,794,50.8,49.2,≤5.0,15.6,12.8,14.0,≤5.0,48.4,8.9,13.2,13.2,15.9,Elementary School,2012-2013,NaN,NaN
2,Bannockburn ES,385,50.9,49.1,≤5.0,8.6,≤5.0,6.5,≤5.0,73.0,9.4,10.9,≤5.0,7.8,Elementary School,2012-2013,NaN,NaN
3,Lucy V. Barnsley ES,663,56.1,43.9,≤5.0,15.5,11.9,26.7,≤5.0,38.9,6.6,15.2,30.5,13.7,Elementary School,2012-2013,NaN,NaN
4,Beall ES,777,52.1,47.9,≤5.0,24.6,14.0,17.1,≤5.0,35.4,8.8,19.3,25.4,7.7,Elementary School,2012-2013,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Springbrook HS,1681,57.9,41.9,≤5.0,10.4,38.4,42.7,≤5.0,5.7,≤5.0,NaN,59.0,7.9,High School,2021-2022,≤5.0,18.6
197,Watkins Mill HS,1594,54.5,45.4,≤5.0,9.8,23.5,58.6,≤5.0,5.3,≤5.0,NaN,67.9,13.4,High School,2021-2022,≤5.0,28.4
198,Wheaton HS,2375,59.2,40.7,≤5.0,11.3,19.6,56.2,≤5.0,10.3,≤5.0,NaN,58.2,9.7,High School,2021-2022,≤5.0,18.4
199,Walt Whitman HS,2029,52.1,47.8,≤5.0,14.3,≤5.0,11.2,≤5.0,63.0,7.5,NaN,≤5.0,9.5,High School,2021-2022,≤5.0,≤5.0


In [128]:
mcps_all.to_csv('mcps_data_new.csv', index=False)